In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Saquarema,BR,-22.9200,-42.5103,84.22,89,8,11.50,clear sky
1,1,Cabo San Lucas,MX,22.8909,-109.9124,85.30,63,100,5.99,overcast clouds
2,2,Mataura,NZ,-46.1927,168.8643,56.48,62,97,4.41,overcast clouds
3,3,Villa Rica,US,33.7321,-84.9191,52.68,39,29,7.11,scattered clouds
4,4,Tiksi,RU,71.6872,128.8694,-22.58,100,95,5.97,overcast clouds


In [3]:
city_data_df.dtypes

City_ID                  int64
City                    object
Country                 object
Lat                    float64
Lng                    float64
Max Temp               float64
Humidity                 int64
Cloudiness               int64
Wind Speed             float64
Current Description     object
dtype: object

In [4]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [5]:
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

In [7]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure()
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp], dissipating=False, max_intensity=300, point_radius=4)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [8]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [9]:
# Heatmap of percent cloudiness
locations = city_data_df[["Lat", "Lng"]]
clouds = city_data_df["Cloudiness"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=clouds, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.

fig

Figure(layout=FigureLayout(height='420px'))

In [11]:
# Ask the customer to add a minimum and maximum temperature value. 
# If you have to do this again "75" & "90"
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [12]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Saquarema,BR,-22.9200,-42.5103,84.22,89,8,11.50,clear sky
1,1,Cabo San Lucas,MX,22.8909,-109.9124,85.30,63,100,5.99,overcast clouds
5,5,Poum,NC,-20.2333,164.0167,81.36,78,15,14.16,few clouds
8,8,Puerto Ayora,EC,-0.7393,-90.3518,78.76,88,75,7.00,broken clouds
9,9,Bathsheba,BB,13.2167,-59.5167,83.01,69,40,14.97,scattered clouds
14,14,Hilo,US,19.7297,-155.0900,77.29,69,75,5.75,light rain
15,15,Sahuaripa,MX,29.0500,-109.2333,81.61,23,28,4.29,scattered clouds
18,18,Big Spring,US,32.2504,-101.4787,78.28,20,1,5.01,clear sky
24,24,Hithadhoo,MV,-0.6000,73.0833,81.14,79,68,13.82,broken clouds
27,27,Rikitea,PF,-23.1203,-134.9692,76.57,74,2,21.68,clear sky


In [13]:
preferred_cities_df.count()

City_ID                181
City                   181
Country                181
Lat                    181
Lng                    181
Max Temp               181
Humidity               181
Cloudiness             181
Wind Speed             181
Current Description    181
dtype: int64

In [14]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Saquarema,BR,84.22,-22.9200,-42.5103,
1,Cabo San Lucas,MX,85.30,22.8909,-109.9124,
5,Poum,NC,81.36,-20.2333,164.0167,
8,Puerto Ayora,EC,78.76,-0.7393,-90.3518,
9,Bathsheba,BB,83.01,13.2167,-59.5167,
14,Hilo,US,77.29,19.7297,-155.0900,
15,Sahuaripa,MX,81.61,29.0500,-109.2333,
18,Big Spring,US,78.28,32.2504,-101.4787,
24,Hithadhoo,MV,81.14,-0.6000,73.0833,
27,Rikitea,PF,76.57,-23.1203,-134.9692,


In [15]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [21]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
# Grab the first hotel from the results and store the name.
try:
    hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
except (IndexError):
    print("Hotel not found... skipping.")

In [ ]:
# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig